In [1]:
%pylab inline
import pandas as pd
import numpy as np
from IPython.display import display
import re

Populating the interactive namespace from numpy and matplotlib


# Data

In [2]:
file_name = '/dataset/personal_projects/lsm/20160123_1차 발주_3280.xlsx'
SHEET_NAME = 'original'

In [3]:
def display_time(seconds):
    response = ''
    hours = floor(seconds/int(60*60))
    seconds -= hours*60*60
    if hours >= 1:
        response += '%d시간 ' % hours
    
    minutes = floor(seconds/int(60))
    seconds -= minutes*60
    if minutes >= 1:
        response += '%d분 ' % minutes
    
    response += '%d초' % seconds
    return response


def display_result(n_epoch, n_location, n_product, n_barcode, is_possible=None):
    cost_location = n_location * FINDING_LOCATION_TIME
    cost_product = n_barcode * FINDING_PRODUCT_TIME
    if is_possible is not None:
        print('시뮬레이션 결과:\t', is_possible)
    
    print('차수 횟수:\t %d개' % n_epoch)
    print('길찾기 횟수:\t %d건' % n_location)
    print('상품찾기 횟수:\t %d건' % n_product)
    print('바코드찾기 횟수:\t %d건' % n_barcode)
    print('총 찾기 횟수:\t %d건' % int(n_location + n_barcode))
    print('갈찾기 소요시간:\t', display_time(cost_location))
    print('바코드찾기소요시간:\t', display_time(cost_product) )
    print('총 소요시간:\t', display_time(cost_location + cost_product))    

In [20]:
class LSM(object):
    EMPTY_LOCATION = 'Z1-1'
    
    def __init__(self, DAS=96):
        self.data = None
        self.das_size = DAS
        
    def load(self, file_name, sheet=SHEET_NAME):
        COL_NAMES = ['date', 'order_number', 'brand_code', 'brand_name', 
                     'product', 'property', 'location', 'barcode', 'n', 'price', 
                     'status', 'orderer', 'recipient', 'address', 'postcode', 
                     'contact1', 'contact2', 'comment', 'part_delivery', 'code']
        self.data = pd.read_excel(file_name, 
                                  names=COL_NAMES,
                                  sheetname=sheet)
    
    def order_by_product(self):
        most_products = self.data[['product', 'n']].groupby(by='product').sum()
        most_products = most_products.sort_values(by='n', ascending=False)
        codes = []
        for product in most_products.index:
            for code in self.data[self.data['product'] == product]['code'].values:
                if code not in codes:
                    codes.append(code)
        
        codes = np.array(codes)
        return codes
    
    def preprocess(self, codes):
        
    

lsm = LSM()
lsm.load(file_name)
codes = lsm.order_by_product()
codes

array([2845982, 2849046, 2849187, ..., 2850710, 2851903, 2849023])